In [1]:
import mytools
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon,Point,LineString,MultiPolygon

# 小区和站点对应关系

In [ ]:
f = mytools.othern.file_name_paths('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/1-基础数据/空洞使用的小区信息/各地市计算/')

In [ ]:
data_z = pd.DataFrame()
for name in f:
    data_excel = pd.ExcelFile(name)
    data_id = data_excel.parse('移动小区')
    data_pz = data_excel.parse('移动物理站')
    data_pz_use = data_pz[[ '小区中文名', 'FDD900_p', 'FDD1800_p', 'F_p', 'D_p', 'E_p', 'A_p']]
    data_pz_use['站点对应小区'] = data_pz_use[[ 'FDD900_p', 'FDD1800_p', 'F_p', 'D_p', 'E_p', 'A_p']].apply(
    lambda x: '/'.join([str(i) for i in list(x) if pd.notnull(i)]),axis=1)
    data_res = data_pz_use.join(data_pz_use['站点对应小区'].str.split(
        '/',expand=True).stack().reset_index(level=1,drop=True).rename('对应编号'))
    data_id['id'] = data_id.index+2
    data_id_use = data_id[['小区中文名', 'id']].rename(columns={'小区中文名':'小区名'})
    data_res['对应编号'] = pd.to_numeric(data_res['对应编号'])
    data_id_use['id'] = data_id_use['id'].astype('object')
    data_res2 = data_res.merge(data_id_use,how='left',left_on='对应编号',right_on='id')
    data_res2.columns = ['站点名称', 'FDD900_p', 'FDD1800_p', 'F_p', 'D_p', 'E_p', 'A_p', '站点对应小区',
           '对应编号', '小区名', 'id']
    data_res2.to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/2-输出数据/'+name.split('/')[-1].split('.')[0]+'.csv',encoding='gbk',index=False)
    print(name,'输出成功')
    data_z = data_z.append(data_res2)
data_z.to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/2-输出数据/汇总.csv',encoding='gbk',index=False)
print('输出完成')

In [3]:
data_z=pd.read_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/2-输出数据/汇总.csv',encoding='gbk')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 小区单独覆盖区域计算

In [14]:
gongcan_s = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/修正后覆盖半径的扇区图层_1原始.data')

In [5]:
# gongcan_s[[ '小区CGI(*)', '小区中文名', '所属地区', 'RRU经度', 'RRU纬度', '天线挂高',
#        '方位角', '工作频段', '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离',
#        '30米覆盖模型距离', '使用覆盖半径', 'distance_use', 'distance_yb', 'h_use', '高度建筑比',
#        'distance_use_3']].to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/1-基础数据/空洞使用的小区信息/kd_use_data.csv',encoding='gbk',index=False)

In [15]:
gongcan_s = gongcan_s[[ '小区CGI(*)', '小区中文名', '所属地区', 'RRU经度', 'RRU纬度', '天线挂高',
       '方位角', '工作频段', '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离',
       '30米覆盖模型距离', '使用覆盖半径', 'distance_use', 'distance_yb', 'h_use', '高度建筑比',
       'distance_use_3', 'geometry']]

In [18]:
gongcan_s = gongcan_s.loc[gongcan_s['归属区县']=='东湖高新开发区']

In [19]:
gongcan_s.shape

(9283, 21)

In [20]:
def fc(x):
    lon = x['RRU经度']
    lat = x['RRU纬度']
    #筛选范围内的所有扇区
    gongcan_s_temp = gongcan_s.loc[(gongcan_s['RRU经度']>(lon-0.05) )&(gongcan_s['RRU经度']<(lon+0.05) ) & 
                  (gongcan_s['RRU纬度']>(lat-0.05) )&(gongcan_s['RRU纬度']<(lat+0.05))]
    #剔除掉共站址的扇区
    b = data_z.loc[data_z['小区名']==x['小区中文名'],'站点名称']
    if len(list(b.values)) == 0 :
        geometry='无效数据'
    else:
        use_s = gongcan_s_temp.loc[~gongcan_s_temp['小区中文名'].isin(list(data_z.loc[data_z['站点名称']==b.values[0],'小区名'].values))]
        if use_s.shape[0]<1:
            geometry='孤站'
        else:
            use_s['省份'] = 'hb'
            use_s1 = use_s.dissolve(by = '省份')
            use_s1['geometry'] = use_s1.buffer(0.0000001)
            geometry = x['geometry'].difference(use_s1['geometry'][0])
            if geometry.area==0:
                geometry=1
    return geometry

In [ ]:
gongcan_s['ji_geo'] = gongcan_s.apply(fc,axis=1)#gongcan_s.iloc[36160:36177,]

In [22]:
mytools.sound()

# 面积计算

In [54]:
gongcan_s_area = mytools.gisn.add_area(gongcan_s,'自身面积')

In [55]:
gongcan_s_area_1 = gongcan_s_area.loc[~((gongcan_s_area['ji_geo']==1) | (gongcan_s_area['ji_geo']=='无效数据') | (gongcan_s_area['ji_geo']=='孤站')) ]

In [56]:
gongcan_s_area_2 = gongcan_s_area.loc[(gongcan_s_area['ji_geo']==1) | (gongcan_s_area['ji_geo']=='无效数据') | (gongcan_s_area['ji_geo']=='孤站') ]

In [57]:
gongcan_s_area_2['单独覆盖区域面积'] = gongcan_s_area_2['ji_geo']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
gongcan_s_area_1 = gongcan_s_area_1.set_geometry('ji_geo')

In [59]:
gongcan_s_area_1 = mytools.gisn.add_area(gongcan_s_area_1,'单独覆盖区域面积')

In [60]:
res = gongcan_s_area_1.append(gongcan_s_area_2)

In [61]:
res.drop(columns=['geometry','ji_geo']).to_csv(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/3-输出结果/每个小区的单独覆盖面积.csv',encoding='gbk',index=False)

# 其他计算

In [63]:
zm_xd = pd.read_excel(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/临时需求，将主城区一般城区与农村小区覆盖有交集的删掉/东新小区覆盖为0.xlsx',
    encoding='gbk')

In [68]:
zm_xd.shape

(3067, 15)

In [71]:
zm_xd_use = zm_xd.merge(gongcan_s[['小区CGI(*)','小区中文名','geometry']],how='left',on=['小区CGI(*)','小区中文名'])

In [76]:
gongcan_nc = gongcan_s.loc[gongcan_s['区域类型']=='农村']

In [80]:
gongcan_nc = gongcan_nc.dissolve(by='所属地区')

In [84]:
zm_xd_use_gdf = gpd.GeoDataFrame(zm_xd_use,crs={"init": "epsg:4326"} ,geometry='geometry')

In [85]:
res = gpd.sjoin(zm_xd_use_gdf,gongcan_nc)

In [86]:
res.drop(columns='geometry').to_csv('G:/1-规划/评估/覆盖空洞/20200712第四版本/9-评估搬迁站点/临时需求，将主城区一般城区与农村小区覆盖有交集的删掉/res.csv',encoding='gbk')